In [1]:
# import matplotlib.pyplot as plt
# import matplotlib.cm as cm
import numpy as np
import pandas as pd


# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

2024-05-22 08:40:19.304129: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


### DNN Regression
It is now time to implements single-input and multiple-inputs DNN models

In [3]:
print('---------------------------------------------------------------------------------')
print('Initializing DNN program')
print('---------------------------------------------------------------------------------')
print('')
#---------------------------------------------------------------------
# Importing Libraries

print('---------------------------------------------------------------------------------')
print('Importing Libraries')
print('---------------------------------------------------------------------------------')
print('')

import time
import sklearn
import tensorflow as tf
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math
#import keras
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM
#from keras.layers import Dropout
#from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
#from keras.callbacks import EarlyStopping
#from keras.preprocessing import sequence
#from keras.utils import pad_sequences
#from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import shap
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
np.random.seed(0)

#---------------------------------------------------------------------
# Defining metric equations

print('---------------------------------------------------------------------------------')
print('Defining Metric Equations')
print('---------------------------------------------------------------------------------')
print('')

def ACC(TP,TN,FP,FN):
    Acc = (TP+TN)/(TP+FP+FN+TN)
    return Acc
def ACC_2 (TP, FN):
    ac = (TP/(TP+FN))
    return ac
def PRECISION(TP,FP):
    Precision = TP/(TP+FP)
    return Precision
def RECALL(TP,FN):
    Recall = TP/(TP+FN)
    return Recall
def F1(Recall, Precision):
    F1 = 2 * Recall * Precision / (Recall + Precision)
    return F1
def BACC(TP,TN,FP,FN):
    BACC =(TP/(TP+FN)+ TN/(TN+FP))*0.5
    return BACC
def MCC(TP,TN,FP,FN):
    MCC = (TN*TP-FN*FP)/(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**.5)
    return MCC
def AUC_ROC(y_test_bin,y_score):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    auc_avg = 0
    counting = 0
    for i in range(n_classes):
      fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
     # plt.plot(fpr[i], tpr[i], color='darkorange', lw=2)
      #print('AUC for Class {}: {}'.format(i+1, auc(fpr[i], tpr[i])))
      auc_avg += auc(fpr[i], tpr[i])
      counting = i+1
    return auc_avg/counting
#---------------------------------------------------------------------
# Defining features of interest
print('---------------------------------------------------------------------------------')
print('Defining features of interest')
print('---------------------------------------------------------------------------------')
print('')

# req_cols = [ ' Packet Length Std', ' Total Length of Bwd Packets', ' Subflow Bwd Bytes',
# ' Destination Port', ' Packet Length Variance', ' Bwd Packet Length Mean',' Avg Bwd Segment Size',
# 'Bwd Packet Length Max', ' Init_Win_bytes_backward','Total Length of Fwd Packets',
# ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' Average Packet Size', ' Packet Length Mean',
# ' Max Packet Length',' Label']

# req_cols = [' Destination Port',' Flow Duration',' Total Fwd Packets',' Total Backward Packets','Total Length of Fwd Packets',' Total Length of Bwd Packets',' Fwd Packet Length Max',' Fwd Packet Length Min',' Fwd Packet Length Mean',' Fwd Packet Length Std','Bwd Packet Length Max',' Bwd Packet Length Min',' Bwd Packet Length Mean',' Bwd Packet Length Std','Flow Bytes/s',' Flow Packets/s',' Flow IAT Mean',' Flow IAT Std',' Flow IAT Max',' Flow IAT Min','Fwd IAT Total',' Fwd IAT Mean',' Fwd IAT Std',' Fwd IAT Max',' Fwd IAT Min','Bwd IAT Total',' Bwd IAT Mean',' Bwd IAT Std',' Bwd IAT Max',' Bwd IAT Min','Fwd PSH Flags',' Bwd PSH Flags',' Fwd URG Flags',' Bwd URG Flags',' Fwd Header Length',' Bwd Header Length','Fwd Packets/s',' Bwd Packets/s',' Min Packet Length',' Max Packet Length',' Packet Length Mean',' Packet Length Std',' Packet Length Variance','FIN Flag Count',' SYN Flag Count',' RST Flag Count',' PSH Flag Count',' ACK Flag Count',' URG Flag Count',' CWE Flag Count',' ECE Flag Count',' Down/Up Ratio',' Average Packet Size',' Avg Fwd Segment Size',' Avg Bwd Segment Size',' Fwd Header Length','Fwd Avg Bytes/Bulk',' Fwd Avg Packets/Bulk',' Fwd Avg Bulk Rate',' Bwd Avg Bytes/Bulk',' Bwd Avg Packets/Bulk','Bwd Avg Bulk Rate','Subflow Fwd Packets',' Subflow Fwd Bytes',' Subflow Bwd Packets',' Subflow Bwd Bytes','Init_Win_bytes_forward',' Init_Win_bytes_backward',' act_data_pkt_fwd',' min_seg_size_forward','Active Mean',' Active Std',' Active Max',' Active Min','Idle Mean',' Idle Std',' Idle Max',' Idle Min',' Label']

# req_cols = [' Down/Up Ratio', ' Fwd URG Flags', ' Flow IAT Std', 'Subflow Fwd Packets', ' Flow Packets/s', ' URG Flag Count', 'FIN Flag Count', ' Bwd Packets/s', 'Bwd Avg Bulk Rate', ' act_data_pkt_fwd', ' Fwd Packet Length Std', ' Bwd Avg Bytes/Bulk', ' Active Max', ' Flow IAT Max', ' min_seg_size_forward', ' Bwd Packet Length Std', ' Fwd IAT Std', ' Fwd Avg Bulk Rate', ' Fwd Packet Length Mean', ' Fwd Packet Length Max', ' Idle Std', ' CWE Flag Count', 'Fwd IAT Total', ' ACK Flag Count', ' Bwd URG Flags', ' Flow IAT Min', ' Flow IAT Mean', ' Total Backward Packets', ' Fwd Avg Packets/Bulk', 'Fwd Avg Bytes/Bulk', ' SYN Flag Count', ' Min Packet Length', ' Fwd Packet Length Min', 'Idle Mean', 'Fwd PSH Flags', ' Fwd IAT Min', ' Fwd Header Length', ' RST Flag Count', ' Idle Max', ' PSH Flag Count', ' Bwd Header Length', ' ECE Flag Count', ' Subflow Bwd Packets', 'Active Mean', 'Flow Bytes/s', ' Bwd IAT Mean', ' Avg Fwd Segment Size', ' Bwd Packet Length Min', ' Active Std', ' Bwd IAT Min', ' Flow Duration', 'Fwd Packets/s', ' Fwd IAT Max', 'Bwd IAT Total', ' Idle Min', ' Bwd PSH Flags', ' Bwd Avg Packets/Bulk', ' Total Fwd Packets', ' Active Min', ' Bwd IAT Std', ' Fwd IAT Mean', ' Bwd IAT Max', ' Label']

req_cols = [
    
    ' Packet Length Std', ' Total Length of Bwd Packets', ' Subflow Bwd Bytes',
    ' Destination Port', ' Packet Length Variance', ' Bwd Packet Length Mean',' Avg Bwd Segment Size',
    'Bwd Packet Length Max', ' Init_Win_bytes_backward','Total Length of Fwd Packets',
    ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' Average Packet Size', ' Packet Length Mean',
    ' Max Packet Length',
    ' Down/Up Ratio', ' Fwd URG Flags', ' Flow IAT Std', 'Subflow Fwd Packets', ' Flow Packets/s', ' URG Flag Count', 'FIN Flag Count', ' Bwd Packets/s', 'Bwd Avg Bulk Rate'
    , ' act_data_pkt_fwd', ' Fwd Packet Length Std', ' Bwd Avg Bytes/Bulk', ' Active Max', ' Flow IAT Max', ' min_seg_size_forward', ' Bwd Packet Length Std', ' Fwd IAT Std', ' Fwd Avg Bulk Rate', ' Fwd Packet Length Mean', ' Fwd Packet Length Max', ' Idle Std', ' CWE Flag Count', 'Fwd IAT Total'
    
    , ' ACK Flag Count', ' Bwd URG Flags', ' Flow IAT Min', ' Flow IAT Mean', ' Total Backward Packets', ' Fwd Avg Packets/Bulk', 'Fwd Avg Bytes/Bulk', ' SYN Flag Count', ' Min Packet Length', ' Fwd Packet Length Min', 'Idle Mean', 'Fwd PSH Flags', ' Fwd IAT Min'
     
    ,  ' Fwd Header Length', ' RST Flag Count', ' Idle Max', ' PSH Flag Count', ' Bwd Header Length', ' ECE Flag Count', ' Subflow Bwd Packets', 'Active Mean', 'Flow Bytes/s', ' Bwd IAT Mean', ' Avg Fwd Segment Size', ' Bwd Packet Length Min', ' Active Std', ' Bwd IAT Min', ' Flow Duration', 'Fwd Packets/s', ' Fwd IAT Max', 'Bwd IAT Total', ' Idle Min', ' Bwd PSH Flags', ' Bwd Avg Packets/Bulk', ' Total Fwd Packets', ' Active Min', ' Bwd IAT Std', ' Fwd IAT Mean', ' Bwd IAT Max'
            
            , ' Label']

# Information gain top 10 features
top10 = [' Average Packet Size',
          ' Packet Length Std', 
          ' Packet Length Variance', 
          ' Packet Length Mean',
            ' Destination Port', 
            ' Subflow Bwd Bytes', 
            ' Total Length of Bwd Packets', 
            ' Avg Bwd Segment Size', 
            ' Bwd Packet Length Mean',  
            'Bwd Packet Length Max', 
            ' Label']



# req_cols = top10

# req_cols = [' Destination Port',' Flow Duration',' Total Fwd Packets',' Total Backward Packets','Total Length of Fwd Packets',' Total Length of Bwd Packets',' Label']
#---------------------------------------------------------------------
#Load Databases from csv file

path_str = '/home/oarreche@ads.iu.edu/HITL/cicids/cicids_db/'
fraction = 1
#---------------------------------------------------------------------
#Load Databases from csv file
print('---------------------------------------------------------------------------------')
print('Loading Databases')
print('---------------------------------------------------------------------------------')
print('')


df0 = pd.read_csv (path_str + 'Wednesday-workingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)

df1 = pd.read_csv (path_str + 'Tuesday-WorkingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df2 = pd.read_csv (path_str +'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df3 = pd.read_csv (path_str +'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df4 = pd.read_csv (path_str +'Monday-WorkingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df5 = pd.read_csv (path_str +'Friday-WorkingHours-Morning.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df6 = pd.read_csv (path_str +'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df7 = pd.read_csv (path_str +'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)

frames = [df0, df1, df2, df3, df4, df5, df6, df7]

df = pd.concat(frames,ignore_index=True)

df = df.sample(frac = 0.333)



# IG popping unwandted columns descriptive accuracy

# df.pop(' min_seg_size_forward')
# df.pop(' PSH Flag Count')
# df.pop(' ACK Flag Count')
# df.pop('Init_Win_bytes_forward')
# df.pop(' Destination Port')

# df.pop(' Bwd Packet Length Mean')
# df.pop('Fwd Packets/s')
# df.pop(' Init_Win_bytes_backward')
# df.pop(' Packet Length Mean')
# df.pop(' Packet Length Std')

# df.pop(' Bwd Packet Length Std')
# df.pop(' Avg Bwd Segment Size')
# df.pop(' Packet Length Variance')
# df.pop(' Flow Duration')
# df.pop(' Max Packet Length')
# df.pop(' Flow IAT Max')
# df.pop(' Idle Max')
# df.pop('Bwd Packet Length Max')
# df.pop(' Fwd IAT Max')
# df.pop(' URG Flag Count')
# df.pop('Fwd PSH Flags')

# df.pop(' Min Packet Length')
# df.pop('Idle Mean')
# df.pop(' Fwd Packet Length Min')
# df.pop(' Average Packet Size')
# df.pop(' Idle Min')
# df.pop('Bwd IAT Total')
# df.pop('Fwd IAT Total')
# df.pop(' Fwd Packet Length Max')
# df.pop(' Avg Fwd Segment Size')
# df.pop(' SYN Flag Count')
# df.pop(' Fwd Packet Length Mean')
# df.pop(' Flow IAT Std')
# df.pop(' Down/Up Ratio')
# df.pop(' Fwd IAT Mean')
# df.pop('FIN Flag Count')
# df.pop(' Bwd Packets/s')
# df.pop(' Fwd Packet Length Std')
# df.pop(' Flow IAT Mean')
# df.pop(' Bwd Packet Length Min')
# df.pop(' Fwd IAT Std')
# df.pop(' Bwd IAT Mean')

# df.pop(' Bwd IAT Std')
# df.pop(' Bwd IAT Max')
# df.pop(' Bwd IAT Min')
# df.pop(' Fwd IAT Min')
# df.pop(' Idle Std')
# df.pop(' Active Max')
# df.pop(' RST Flag Count')
# df.pop(' Subflow Fwd Bytes')
# df.pop(' ECE Flag Count')
# df.pop(' Flow IAT Min')
# df.pop('Total Length of Fwd Packets')
# df.pop(' Active Std')
# df.pop(' Active Min')
# df.pop(' Bwd Header Length')
# df.pop(' Fwd Header Length')
# df.pop('Active Mean')
# df.pop(' Total Fwd Packets')
# df.pop(' Total Backward Packets')
# df.pop('Subflow Fwd Packets')
# df.pop(' act_data_pkt_fwd')
# df.pop(' Subflow Bwd Bytes')
# df.pop(' Fwd URG Flags')
# df.pop(' CWE Flag Count')
# df.pop(' Subflow Bwd Packets')
# df.pop(' Total Length of Bwd Packets')
# df.pop('Flow Bytes/s')
# df.pop(' Flow Packets/s')
# df.pop(' Bwd PSH Flags')
# df.pop(' Bwd URG Flags')
# df.pop('Fwd Avg Bytes/Bulk')
# df.pop(' Fwd Avg Packets/Bulk')
# df.pop(' Fwd Avg Bulk Rate')
# df.pop(' Bwd Avg Bytes/Bulk')
# df.pop(' Bwd Avg Packets/Bulk')
# df.pop('Bwd Avg Bulk Rate')

# LRP popping unwandted columns descriptive accuracy


# df.pop(' min_seg_size_forward')
# df.pop(' PSH Flag Count')
# df.pop(' ACK Flag Count')
# df.pop('Init_Win_bytes_forward')
# df.pop(' Destination Port')

# df.pop(' Bwd Packet Length Mean')
# df.pop('Fwd Packets/s')
# df.pop(' Init_Win_bytes_backward')
# df.pop(' Packet Length Mean')
# df.pop(' Packet Length Std')

# df.pop(' Bwd Packet Length Std')
# df.pop(' Avg Bwd Segment Size')
# df.pop(' Packet Length Variance')
# df.pop(' Flow Duration')
# df.pop(' Max Packet Length')
# df.pop(' Flow IAT Max')
# df.pop(' Idle Max')
# df.pop('Bwd Packet Length Max')
# df.pop(' Fwd IAT Max')
# df.pop(' URG Flag Count')
# df.pop('Fwd PSH Flags')

# df.pop(' Min Packet Length')
# df.pop('Idle Mean')
# df.pop(' Fwd Packet Length Min')
# df.pop(' Average Packet Size')
# df.pop(' Idle Min')
# df.pop('Bwd IAT Total')
# df.pop('Fwd IAT Total')
# df.pop(' Fwd Packet Length Max')
# df.pop(' Avg Fwd Segment Size')
# df.pop(' SYN Flag Count')
# df.pop(' Fwd Packet Length Mean')
# df.pop(' Flow IAT Std')
# df.pop(' Down/Up Ratio')
# df.pop(' Fwd IAT Mean')
# df.pop('FIN Flag Count')
# df.pop(' Bwd Packets/s')
# df.pop(' Fwd Packet Length Std')
# df.pop(' Flow IAT Mean')
# df.pop(' Bwd Packet Length Min')
# df.pop(' Fwd IAT Std')
# df.pop(' Bwd IAT Mean')
# df.pop(' Bwd IAT Std')

# df.pop(' Bwd IAT Max')
# df.pop(' Bwd IAT Min')
# df.pop(' Fwd IAT Min')
# df.pop(' Idle Std')
# df.pop(' Active Max')
# df.pop(' RST Flag Count')
# df.pop(' Subflow Fwd Bytes')
# df.pop(' ECE Flag Count')
# df.pop(' Flow IAT Min')
# df.pop('Total Length of Fwd Packets')
# df.pop(' Active Std')
# df.pop(' Active Min')
# df.pop(' Bwd Header Length')
# df.pop(' Fwd Header Length')
# df.pop('Active Mean')
# df.pop(' Total Fwd Packets')
# df.pop(' Total Backward Packets')
# df.pop('Subflow Fwd Packets')
# df.pop(' act_data_pkt_fwd')
# df.pop(' Subflow Bwd Bytes')
# df.pop(' Fwd URG Flags')
# df.pop(' CWE Flag Count')
# df.pop(' Subflow Bwd Packets')
# df.pop(' Total Length of Bwd Packets')
# df.pop('Flow Bytes/s')
# df.pop(' Flow Packets/s')
# df.pop(' Bwd PSH Flags')
# df.pop(' Bwd URG Flags')
# df.pop('Fwd Avg Bytes/Bulk')
# df.pop(' Fwd Avg Packets/Bulk')
# df.pop(' Fwd Avg Bulk Rate')
# df.pop(' Bwd Avg Bytes/Bulk')
# df.pop(' Bwd Avg Packets/Bulk')
# df.pop('Bwd Avg Bulk Rate')


# DeepLift popping unwandted columns descriptive accuracy

# df.pop(' PSH Flag Count')
# df.pop(' ACK Flag Count')
# df.pop(' Bwd Packet Length Mean')
# df.pop('Init_Win_bytes_forward')
# df.pop(' Bwd Packet Length Std')

# df.pop('Fwd Packets/s')
# df.pop(' Packet Length Std')
# df.pop(' Packet Length Variance')
# df.pop(' Destination Port')
# df.pop(' Avg Bwd Segment Size')

# df.pop(' Packet Length Mean')
# df.pop(' URG Flag Count')
# df.pop(' Idle Max')
# df.pop(' Flow IAT Max')
# df.pop('Fwd PSH Flags')
# df.pop(' Flow Duration')
# df.pop(' Max Packet Length')
# df.pop('Bwd Packet Length Max')
# df.pop(' Fwd IAT Max')
# df.pop('FIN Flag Count')
# df.pop(' min_seg_size_forward')

# df.pop('Idle Mean')
# df.pop(' Idle Min')
# df.pop(' Init_Win_bytes_backward')
# df.pop('Bwd IAT Total')
# df.pop('Fwd IAT Total')
# df.pop(' Average Packet Size')
# df.pop(' Flow IAT Std')
# df.pop(' Min Packet Length')
# df.pop(' SYN Flag Count')
# df.pop(' Fwd Packet Length Min')
# df.pop(' Fwd IAT Mean')
# df.pop(' Bwd Packet Length Min')
# df.pop(' Fwd IAT Std')
# df.pop(' Fwd Packet Length Max')
# df.pop(' Fwd Packet Length Mean')
# df.pop(' Fwd Packet Length Std')
# df.pop(' Flow IAT Mean')
# df.pop(' Avg Fwd Segment Size')
# df.pop(' Bwd IAT Max')
# df.pop(' Bwd IAT Mean')
# df.pop(' Down/Up Ratio')

# df.pop(' Bwd IAT Min')
# df.pop(' Fwd IAT Min')
# df.pop(' Bwd IAT Std')
# df.pop(' Bwd Packets/s')
# df.pop(' Idle Std')
# df.pop(' Active Max')
# df.pop(' Flow IAT Min')
# df.pop(' Subflow Fwd Bytes')
# df.pop('Total Length of Fwd Packets')
# df.pop(' Active Std')
# df.pop(' RST Flag Count')
# df.pop(' Active Min')
# df.pop('Active Mean')
# df.pop(' ECE Flag Count')
# df.pop(' CWE Flag Count')
# df.pop(' Fwd URG Flags')
# df.pop(' Total Backward Packets')
# df.pop(' act_data_pkt_fwd')
# df.pop(' Total Fwd Packets')
# df.pop(' Subflow Bwd Bytes')
# df.pop('Subflow Fwd Packets')
# df.pop(' Fwd Header Length')
# df.pop(' Bwd Header Length')
# df.pop(' Subflow Bwd Packets')
# df.pop(' Total Length of Bwd Packets')
# df.pop('Bwd Avg Bulk Rate')
# df.pop(' Bwd Avg Packets/Bulk')
# df.pop(' Bwd Avg Bytes/Bulk')
# df.pop(' Fwd Avg Bulk Rate')
# df.pop(' Fwd Avg Packets/Bulk')
# df.pop(' Bwd PSH Flags')
# df.pop('Flow Bytes/s')
# df.pop(' Flow Packets/s')
# df.pop(' Bwd URG Flags')
# df.pop('Fwd Avg Bytes/Bulk')

#---------------------------------------------------------------------
# Normalize database
print('---------------------------------------------------------------------------------')
print('Normalizing database')
print('---------------------------------------------------------------------------------')
print('')

df_max_scaled = df.copy()

y = df_max_scaled[' Label'].replace({'DDoS' :'Dos/Ddos' ,'DoS GoldenEye': 'Dos/Ddos', 'DoS Hulk': 'Dos/Ddos', 'DoS Slowhttptest': 'Dos/Ddos', 'DoS slowloris': 'Dos/Ddos', 'Heartbleed': 'Dos/Ddos','FTP-Patator': 'Brute Force', 'SSH-Patator': 'Brute Force','Web Attack - Brute Force': 'Web Attack', 'Web Attack - Sql Injection': 'Web Attack', 'Web Attack - XSS': 'Web Attack'})

df_max_scaled.pop(' Label')


df_max_scaled
for col in df_max_scaled.columns:
    t = abs(df_max_scaled[col].max())
    df_max_scaled[col] = df_max_scaled[col]/t
df_max_scaled
df = df_max_scaled.assign( Label = y)
#df
df = df.fillna(0)

#---------------------------------------------------------------------

# Separate features and labels 
print('---------------------------------------------------------------------------------')
print('Separating features and labels')
print('---------------------------------------------------------------------------------')
print('')

y = df.pop('Label')
X = df
# summarize class distribution
counter = Counter(y)
print(counter)
# transform the dataset
# print('---------------------------------------------------------------------------------')
# result_list = [counter['None'],counter['Denial of Service'], counter['Port Scanning']]
# print('number of Labels  ',result_list)
print('---------------------------------------------------------------------------------')

# # Create an instance of RandomUnderSampler
# rus = RandomUnderSampler()

# # Balance the dataset using RandomUnderSampler
# X_resampled, y_resampled = rus.fit_resample(X, y)

# # Create an instance of SMOTE
# smote = SMOTE()

# # Balance the dataset using SMOTE
# X_resampled, y_resampled = smote.fit_resample(X, y)

# X = X_resampled
# y = y_resampled

# summarize class distribution
counter = Counter(y)
print(counter)

df = X.assign( Label = y)

y, label = pd.factorize(y)
# y_test, label = pd.factorize(test['Label'])
#---------------------------------------------------------------------

# Separate Training and Testing db
print('---------------------------------------------------------------------------------')
print('Separating Training and Testing db')
print('---------------------------------------------------------------------------------')
print('')

X_train,X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=0.7,random_state=42)
df = X.assign( Label = y)




---------------------------------------------------------------------------------
Initializing DNN program
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Importing Libraries
---------------------------------------------------------------------------------



/home/oarreche@ads.iu.edu/anaconda3/envs/tf23/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---------------------------------------------------------------------------------
Defining Metric Equations
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Defining features of interest
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Loading Databases
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Normalizing database
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Separating features and labels
---------------------------------------------------------------------------------

Counter({'BENIGN': 756652, 'Dos/Ddos': 127047, 'P

In [4]:
import numpy as np

import tensorflow as tf

# Now you can use Keras modules directly from tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten

# import keras
# from keras.models import Sequential
# from keras.layers import Dense, Flatten
import innvestigate
tf.compat.v1.disable_eager_execution()

In [5]:
from tensorflow.keras.callbacks import LearningRateScheduler

print('---------------------------------------------------------------------------------')
print('Defining the DNN model')
print('---------------------------------------------------------------------------------')
print('')

# dropout_rate = 0.01
nodes = 7

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(len(X_train.columns,))))

model.add(tf.keras.layers.Dense(nodes, activation='relu'))

# model.add(tf.keras.layers.Dense(nodes, activation='relu'))
# model.add(tf.keras.layers.Dense(nodes, activation='relu'))
# model.add(tf.keras.layers.Dense(nodes, activation='relu'))
# model.add(tf.keras.layers.Dense(nodes, activation='relu'))
# model.add(tf.keras.layers.Dense(nodes, activation='relu'))




model.add(tf.keras.layers.Dense(7))
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


#---------------------------------------------------------------------

# # Define your DNN model
# model = Sequential([
#     Flatten(input_shape=(28, 28)),  # Input layer
#     Dense(128, activation='relu'),  # Hidden layer
#     Dense(10, activation='relu')  # Output layer
# ])

# # Compile the model
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])



# # Save the trained model
# model.save('your_model.h5')

# # Load your trained model
# model = tf.keras.models.load_model('your_model.h5')


---------------------------------------------------------------------------------
Defining the DNN model
---------------------------------------------------------------------------------



In [6]:
from tensorflow.keras.models import load_model

from tensorflow.keras.callbacks import EarlyStopping
# Define your learning rate schedule function
# def lr_schedule(epoch):
#     # Your learning rate schedule logic here
#     learning_rate = 0.1
#     if epoch > 10:
#         learning_rate = 0.01
#     if epoch > 20:
#         learning_rate = 0.001
#     return learning_rate
# lr_sched = LearningRateScheduler(lambda epoch: 1e-3 * (0.75 ** np.floor(epoch / 2)))

# # Create a LearningRateScheduler callback
# lr_scheduler = LearningRateScheduler(lr_schedule)
print('---------------------------------------------------------------------------------')
print('Training the model')
print('---------------------------------------------------------------------------------')
print('')

start = time.time()

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)

# Modify model.fit to include the EarlyStopping callback
model.fit(X_train, y_train, epochs=1000, batch_size=len(X_train), callbacks=[early_stopping])

# Pass the lr_scheduler callback to your model.fit() function
# model.fit(X_train, y_train, callbacks=[lr_scheduler], ...)


end = time.time()
print('---------------------------------------------------------------------------------')
print('ELAPSE TIME TRAINING MODEL: ',(end - start)/60, 'min')
print('---------------------------------------------------------------------------------')
print('')


---------------------------------------------------------------------------------
Training the model
---------------------------------------------------------------------------------

Train on 659845 samples
Epoch 1/1000


2024-05-22 08:40:39.600307: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-05-22 08:40:39.606751: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:67:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.54GiB deviceMemoryBandwidth: 715.34GiB/s
2024-05-22 08:40:39.606898: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:68:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2024-05-22 08:40:39.606920: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-05-22 08:40:39.622463: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2024-05

659845/659845 [==============================] - 0s 1us/sample - loss: 8.6109 - accuracy: 0.0727
Epoch 2/1000
659845/659845 [==============================] - 0s 1us/sample - loss: 3.7405 - accuracy: 0.3299
Epoch 3/1000
659845/659845 [==============================] - 0s 1us/sample - loss: 2.8344 - accuracy: 0.6948
Epoch 4/1000
659845/659845 [==============================] - 0s 1us/sample - loss: 2.5269 - accuracy: 0.7557
Epoch 5/1000
659845/659845 [==============================] - 0s 1us/sample - loss: 2.1289 - accuracy: 0.7842
Epoch 6/1000
659845/659845 [==============================] - 0s 1us/sample - loss: 1.5345 - accuracy: 0.8008
Epoch 7/1000
659845/659845 [==============================] - 0s 1us/sample - loss: 1.0826 - accuracy: 0.8022
Epoch 8/1000
659845/659845 [==============================] - 0s 1us/sample - loss: 0.7268 - accuracy: 0.8027
Epoch 9/1000
659845/659845 [==============================] - 0s 1us/sample - loss: 0.6967 - accuracy: 0.8028
Epoch 10/1000
659845/65

In [7]:

print('---------------------------------------------------------------------------------')
print('Model Prediction')
print('---------------------------------------------------------------------------------')
print('')
print('---------------------------------------------------------------------------------')
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print('ELAPSE TIME MODEL PREDICTION: ',(end - start)/60, 'min')
print('---------------------------------------------------------------------------------')
print('')

#print(y_pred)
ynew = np.argmax(y_pred,axis = 1)
#print(ynew)
score = model.evaluate(X_test, y_test,verbose=1)
#print(score)
pred_label = label[ynew]
#print(score)

#---------------------------------------------------------------------
# pd.crosstab(test['ALERT'], preds, rownames=['Actual ALERT'], colnames = ['Predicted ALERT'])

from sklearn.metrics import accuracy_score

accuracy =accuracy_score(y_test, ynew)*100
print(accuracy)

from collections import Counter

label_counts = Counter(y_test)
print(label_counts)

label_counts = Counter(ynew)
print(label_counts)

accuracy =accuracy_score(y_test, ynew)*100
print(accuracy)

---------------------------------------------------------------------------------
Model Prediction
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------


`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


ELAPSE TIME MODEL PREDICTION:  0.07332231998443603 min
---------------------------------------------------------------------------------

80.2352966137656
Counter({0: 226902, 2: 38054, 1: 16073, 3: 1351, 5: 205, 4: 202, 6: 5})
Counter({0: 282789, 1: 3})
80.2352966137656


## Integrated Gradients


In [8]:
single_sample_df = X_test.iloc[[0]]

In [9]:
single_sample_df

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
2255405,0.000809,0.000775,0.000009,0.000007,0.000055,5.990626e-07,0.001926,0.021792,0.009696,0.0,...,0.000005,0.231884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
%%time
# Create an analyzer for the model
analyzer = innvestigate.create_analyzer("integrated_gradients", model)

# Replace X_test with your input data

# X_test = np.random.rand(100, 28, 28)  # Example input data

# X_test2 = X_test.frac(0.001, random_state=42)

# Perform LRP analysis on the input data
analysis = analyzer.analyze(X_test)

# Perform LRP analysis on a certain number of samples
# analysis = analyzer.analyze(X_test.sample(10000))

#uncomment for single sample
# analysis = analyzer.analyze(single_sample_df)

# Print or use the analysis results as needed
print(analysis)

print(len(X_test))
print(len(X_test.columns))
names = X_test.columns
print(analysis.shape)
print(type(analysis))
scores = pd.DataFrame(analysis)
print(analysis)
scores_abs = scores.abs()

# Calculate the sum of each column
sum_of_columns = scores_abs.sum(axis=0)

names = list(names)

names

sum_of_columns = list(sum_of_columns)

sum_of_columns
# type(sum_of_columns)
# # sorted_series = sum_of_columns.sort_values(ascending=False)

# print("Sorted Series in descending order:")
# print(sorted_series)
### Results
# names = ['John', 'Alice', 'Bob', 'Emily']
# sum_of_columns = [10, 5, 15, 8]

# Zip the two lists together
combined = list(zip(names, sum_of_columns))

# Sort the combined list in descending order based on the values from sum_of_columns
sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)

# Unzip the sorted_combined list to separate names and sum_of_columns
sorted_names, sorted_sum_of_columns = zip(*sorted_combined)

print(sorted_names)
print(sorted_sum_of_columns)

sorted_names


[[0.    0.03  0.    ... 0.    0.24  0.123]
 [0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.031 0.    ... 0.    0.243 0.124]
 ...
 [0.002 0.    0.    ... 0.    0.    0.   ]
 [0.256 0.    0.    ... 0.    0.    0.   ]
 [0.    0.02  0.    ... 0.    0.029 0.015]]
282792
77
(10000, 77)
<class 'numpy.ndarray'>
[[0.    0.03  0.    ... 0.    0.24  0.123]
 [0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.031 0.    ... 0.    0.243 0.124]
 ...
 [0.002 0.    0.    ... 0.    0.    0.   ]
 [0.256 0.    0.    ... 0.    0.    0.   ]
 [0.    0.02  0.    ... 0.    0.029 0.015]]
(' ACK Flag Count', ' PSH Flag Count', ' Destination Port', ' Idle Max', 'Idle Mean', ' Avg Bwd Segment Size', ' URG Flag Count', ' Bwd Packet Length Mean', ' Idle Min', 'Fwd PSH Flags', ' Bwd Packet Length Std', 'Fwd IAT Total', ' Fwd IAT Std', ' Packet Length Std', ' Flow Duration', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' Bwd IAT Mean', ' Max Packet Length', ' Packet Length Variance', ' min_seg_size_forwa

(' ACK Flag Count',
 ' PSH Flag Count',
 ' Destination Port',
 ' Idle Max',
 'Idle Mean',
 ' Avg Bwd Segment Size',
 ' URG Flag Count',
 ' Bwd Packet Length Mean',
 ' Idle Min',
 'Fwd PSH Flags',
 ' Bwd Packet Length Std',
 'Fwd IAT Total',
 ' Fwd IAT Std',
 ' Packet Length Std',
 ' Flow Duration',
 'Init_Win_bytes_forward',
 ' Init_Win_bytes_backward',
 ' Bwd IAT Mean',
 ' Max Packet Length',
 ' Packet Length Variance',
 ' min_seg_size_forward',
 ' Fwd Packet Length Mean',
 ' Flow IAT Std',
 ' Fwd IAT Mean',
 'Bwd Packet Length Max',
 ' Fwd Packet Length Max',
 ' Avg Fwd Segment Size',
 ' SYN Flag Count',
 ' Fwd IAT Max',
 'Fwd Packets/s',
 ' Flow IAT Mean',
 ' Bwd IAT Min',
 'Bwd IAT Total',
 ' Average Packet Size',
 'FIN Flag Count',
 ' Packet Length Mean',
 ' Bwd IAT Max',
 ' Fwd Packet Length Min',
 ' Flow IAT Max',
 ' Down/Up Ratio',
 ' Min Packet Length',
 ' Idle Std',
 ' Bwd Packets/s',
 ' Fwd IAT Min',
 ' Flow IAT Min',
 ' Bwd IAT Std',
 ' Bwd Packet Length Min',
 ' ECE Flag C

## LRP


In [32]:
%%time

# Create an analyzer for the model
analyzer = innvestigate.create_analyzer("lrp.z", model)

# Perform LRP analysis on the input data
analysis = analyzer.analyze(X_test)

# Perform LRP analysis on a certain number of samples
# analysis = analyzer.analyze(X_test.sample(10000))

#uncomment for single sample
# analysis = analyzer.analyze(single_sample_df)

# Print or use the analysis results as needed
print(analysis)
print(len(X_test))
print(len(X_test.columns))
names = X_test.columns
print(analysis.shape)
print(type(analysis))
scores = pd.DataFrame(analysis)
print(analysis)
scores_abs = scores.abs()

# Calculate the sum of each column
sum_of_columns = scores_abs.sum(axis=0)

names = list(names)

names

sum_of_columns = list(sum_of_columns)

sum_of_columns
# type(sum_of_columns)
# # sorted_series = sum_of_columns.sort_values(ascending=False)

# print("Sorted Series in descending order:")
# print(sorted_series)
# names = ['John', 'Alice', 'Bob', 'Emily']
# sum_of_columns = [10, 5, 15, 8]

# Zip the two lists together
combined = list(zip(names, sum_of_columns))

# Sort the combined list in descending order based on the values from sum_of_columns
sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)

# Unzip the sorted_combined list to separate names and sum_of_columns
sorted_names, sorted_sum_of_columns = zip(*sorted_combined)

print(sorted_names)
print(sorted_sum_of_columns)

sorted_names


[[ 0.301  0.     0.    ...  0.     0.     0.   ]
 [ 0.002  0.     0.    ...  0.     0.     0.   ]
 [ 0.     0.     0.    ...  0.     0.     0.   ]
 ...
 [ 0.002  0.     0.    ...  0.     0.     0.   ]
 [ 0.     0.    -0.    ...  0.     0.     0.   ]
 [ 0.     0.     0.    ...  0.     0.     0.   ]]
282792
77
(10000, 77)
<class 'numpy.ndarray'>
[[ 0.301  0.     0.    ...  0.     0.     0.   ]
 [ 0.002  0.     0.    ...  0.     0.     0.   ]
 [ 0.     0.     0.    ...  0.     0.     0.   ]
 ...
 [ 0.002  0.     0.    ...  0.     0.     0.   ]
 [ 0.     0.    -0.    ...  0.     0.     0.   ]
 [ 0.     0.     0.    ...  0.     0.     0.   ]]
(' ACK Flag Count', ' PSH Flag Count', ' Destination Port', ' Idle Max', 'Idle Mean', ' URG Flag Count', ' Bwd Packet Length Mean', ' Avg Bwd Segment Size', 'Fwd PSH Flags', ' Bwd Packet Length Std', ' Idle Min', 'Fwd IAT Total', ' Packet Length Std', ' min_seg_size_forward', ' Flow Duration', ' Fwd IAT Std', 'Init_Win_bytes_forward', ' Init_Win_bytes_

(' ACK Flag Count',
 ' PSH Flag Count',
 ' Destination Port',
 ' Idle Max',
 'Idle Mean',
 ' URG Flag Count',
 ' Bwd Packet Length Mean',
 ' Avg Bwd Segment Size',
 'Fwd PSH Flags',
 ' Bwd Packet Length Std',
 ' Idle Min',
 'Fwd IAT Total',
 ' Packet Length Std',
 ' min_seg_size_forward',
 ' Flow Duration',
 ' Fwd IAT Std',
 'Init_Win_bytes_forward',
 ' Init_Win_bytes_backward',
 ' Bwd IAT Mean',
 ' Max Packet Length',
 ' Packet Length Variance',
 ' SYN Flag Count',
 'Bwd Packet Length Max',
 ' Flow IAT Std',
 ' Fwd Packet Length Mean',
 ' Fwd IAT Mean',
 'Fwd Packets/s',
 ' Fwd IAT Max',
 ' Fwd Packet Length Max',
 ' Bwd IAT Min',
 ' Avg Fwd Segment Size',
 'Bwd IAT Total',
 ' Flow IAT Mean',
 ' Packet Length Mean',
 'FIN Flag Count',
 ' Average Packet Size',
 ' Flow IAT Max',
 ' Bwd IAT Max',
 ' Fwd Packet Length Min',
 ' Down/Up Ratio',
 ' Idle Std',
 ' Fwd IAT Min',
 ' Bwd Packets/s',
 ' Flow IAT Min',
 ' Min Packet Length',
 ' Bwd Packet Length Min',
 ' Bwd IAT Std',
 ' Fwd Packet

## SHAP


In [27]:
%%time

output_file_name = 'SHAPCIC.txt'
with open(output_file_name, "w") as f:print('',file = f)

###here

print('---------------------------------------------------------------------------------')
print('Generating Explainer')
print('---------------------------------------------------------------------------------')

samples = 2500

# uncomment for single sample
# samples = 1

Label = label
# df.pop('Label')
#train.pop('Label')
test = X_test
train = X_train
#df.pop('is_train')
start_index = 0

#end_index = len(test)
end_index = samples
#test = test[features]
explainer = shap.DeepExplainer(model,train[start_index:end_index].values.astype('float'))
shap_values = explainer.shap_values(test[start_index:end_index].values.astype('float'))
# shap_values = explainer.shap_values(test[start_index:len(test)].values.astype('float'))

# shap.summary_plot(shap_values = shap_values,
#                  features = test[start_index:end_index],
#                   class_names=[label[0],label[1],label[2],label[3],label[4],label[5],label[6]],show=False)

# plt.savefig('DNN_Shap_Summary_Cicids.png')
# plt.clf()

vals= np.abs(shap_values).mean(1)
feature_importance = pd.DataFrame(list(zip(train.columns, sum(vals))), columns=['col_name','feature_importance_vals'])
feature_importance.sort_values(by=['feature_importance_vals'], ascending=False,inplace=True)
feature_importance.head()
print(feature_importance.to_string())



print('---------------------------------------------------------------------------------')
# feature_importance_vals = 'feature_importance_vals'  # Replace with the name of the column you want to extract
feature_val = feature_importance['feature_importance_vals'].tolist()

# col_name = 'col_name'  # Replace with the name of the column you want to extract
feature_name = feature_importance['col_name'].tolist()


# for item1, item2 in zip(feature_name, feature_val):
#     print(item1, item2)


# Use zip to combine the two lists, sort based on list1, and then unzip them
zipped_lists = list(zip(feature_name, feature_val))
zipped_lists.sort(key=lambda x: x[1],reverse=True)

# Convert the sorted result back into separate lists
sorted_list1, sorted_list2 = [list(x) for x in zip(*zipped_lists)]

for k in sorted_list1:
  with open(output_file_name, "a") as f:print("df.pop('",k,"')", sep='',file = f)
with open(output_file_name, "a") as f:print("Trial_ =[", file = f)
print("Trial_ =[")
for k in sorted_list1:
  with open(output_file_name, "a") as f:print("'",k,"',", sep='', file = f)
  print("'",k,"',", sep='')
with open(output_file_name, "a") as f:print("]", file = f)
print('---------------------------------------------------------------------------------')




# # shap.summary_plot(shap_values = shap_values[0],
# #                  features = test[start_index:end_index],
# #                   class_names=[label[0],label[1],label[2],label[3],label[4],label[5],label[6]],show=False)


# # plt.savefig('DNN_Shap_Summary_Beeswarms.png')
# # plt.clf()



# print('---------------------------------------------------------------------------------')
# print('Generating Sparsity Graph')
# print('---------------------------------------------------------------------------------')
# print('')

# # Find the minimum and maximum values in the list
# min_value = min(feature_val)
# max_value = max(feature_val)

# # Normalize the list to the range [0, 1]
# normalized_list = []
# for x in feature_val:
#     if max_value - min_value == 0:
#         normalized_list.append(0)
#     else:
#         normalized_list.append((x - min_value) / (max_value - min_value))
   
# # print(feature_name,normalized_list,'\n')
# # for item1, item2 in zip(feature_name, normalized_list):
# #     print(item1, item2)

# #calculating Sparsity

# # Define the threshold
# threshold = 1e-10

# # Initialize a count variable to keep track of values below the threshold
# count_below_threshold = 0

# # Iterate through the list and count values below the threshold
# for value in normalized_list:
#     if value < threshold:
#         count_below_threshold += 1

# Sparsity = count_below_threshold/len(normalized_list)
# Spar = []
# print('Sparsity = ',Sparsity)
# X_axis = []
# #----------------------------------------------------------------------------
# for i in range(0, 11):
#     i/10
#     threshold = i/10
#     for value in normalized_list:
#         if value < threshold:
#             count_below_threshold += 1

#     Sparsity = count_below_threshold/len(normalized_list)
#     Spar.append(Sparsity)
#     X_axis.append(i/10)
#     count_below_threshold = 0


# #---------------------------------------------------------------------------

# with open(output_file_name, "a") as f:print('y_axis_RF = ', Spar ,'', file = f)
# with open(output_file_name, "a") as f:print('x_axis_RF = ', X_axis ,'', file = f)

# plt.clf()

# # Create a plot
# plt.plot(X_axis, Spar, marker='o', linestyle='-')

# # Set labels for the axes
# plt.xlabel('X-Axis')
# plt.ylabel('Y-Axis')

# # Set the title of the plot
# plt.title('Values vs. X-Axis')

# # Show the plot
# # plt.show()
# plt.savefig('sparsity.png')
# plt.clf()

---------------------------------------------------------------------------------
Generating Explainer
---------------------------------------------------------------------------------


Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


                        col_name  feature_importance_vals
46                PSH Flag Count                 0.211347
0               Destination Port                 0.194768
47                ACK Flag Count                 0.172430
75                      Idle Max                 0.119976
1                  Flow Duration                 0.109994
30                 Fwd PSH Flags                 0.108459
48                URG Flag Count                 0.100754
54          Avg Bwd Segment Size                 0.096816
20                 Fwd IAT Total                 0.093486
65        Init_Win_bytes_forward                 0.086614
73                     Idle Mean                 0.084154
12        Bwd Packet Length Mean                 0.077557
76                      Idle Min                 0.077397
25                 Bwd IAT Total                 0.071244
13         Bwd Packet Length Std                 0.056536
40            Packet Length Mean                 0.056016
23            